In [ ]:
import os
# 한글 지원
os.putenv('NLS_LANG', '.UTF8')

In [ ]:
def ipchecker():    
    import urllib 
    import re
    
    pattern = re.compile(b'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}')
    dyndns = urllib.request.urlopen('http://checkip.dyndns.org').read()
    
    wan = re.search(pattern, dyndns).group().decode('utf-8')
    
    return wan

In [3]:
# base db_connect param

local_ip = '200.1.220.251:1521/xe' # 원내에서 접속할 db_ip
outside_ip = '61.35.152.103:1521/xe' # 원밖에서 접속할 db_ip
localhost = 'localhost:1521/xe'

db_id = 'begopa' 
db_pwd = 'begopa1'

academy_ip = '61.35.152.66'
db_ip = local_ip if academy_ip == ipchecker() else outside_ip

# db_ip = localhost # 집에서 본인의 개인 컴퓨터dptj  오라클에 접속할때는
# db_ip = localhost

In [4]:
import pickle
# load object ::
# :: file_nm
# :: columns and data_type
with open("./pickle/col2dict.pickle","rb") as fr:
    col2dict = pickle.load(fr)
with open("./pickle/table_nm_set.pickle","rb") as fr:
    table_nm_set = pickle.load(fr)

In [5]:
path = "./sg-data/"
desc_path = path + 'desc/'
col_path = path + 'col/'
origin_path = path + 'origin/'

file_list = os.listdir(origin_path)

In [6]:
# _, _, _, table_col_set_ = extract_table_columns_set(file_list[0]) # _, :: 임시로 쓰고 버리는 값.

In [7]:
# 자치구별 상권 변화지표 db 에서 한글 컬럼으로 가져오기
# oracle_to_df('자치구별 상권변화지표')
def load_to_df(kr_tbl_nm):
    for key, value in table_nm_set.items():
        if kr_tbl_nm == value:
            table_nm = key
            columns = [ arr[0] for arr in col2dict[key] ]
           
    import cx_Oracle
    import pandas as pd
    
    db = cx_Oracle.connect(db_id, db_pwd, db_ip) 
        
    sql = 'select * from ' + table_nm

    df = pd.read_sql(sql=sql, con = db) 
    df.columns = columns
    db.close
    
    return df


In [8]:
import re
import os

file_list = os.listdir(origin_path)
for file in file_list:
    kr_tbl_nm = re.findall('(\d\d).*\((.*)\)',file)
    print(kr_tbl_nm[0][0], kr_tbl_nm[0][1])

00 상권-영역
01 상권-추정유동인구
02 상권-집객시설
03 상권배후지-추정유동인구
04 상권배후지-집객시설
05 상권배후지-상주인구
06 상권-상주인구
07 상권배후지-아파트
08 상권배후지-점포
09 상권-상권변화지표
10 행정동별 상권변화지표
11 상권-추정매출
12 상권-직장인구
13 상권배후지-소득소비
14 상권배후지-직장인구
15 자치구별 상권변화지표
16 상권-아파트
18 상권-점포
19 상권배후지-추정매출


In [9]:
import pandas as pd

df = load_to_df('상권배후지-상주인구')
df1 = df.copy()
df1.head()


,기준_년_분기_코드,상권_코드,hl_총_상주인구_수,hl_남성_상주인구_수,hl_여성_상주인구_수,hl_연령대_10_상주인구_수,hl_연령대_20_상주인구_수,hl_연령대_30_상주인구_수,hl_연령대_40_상주인구_수,hl_연령대_50_상주인구_수,...,hl_남성연령대_60_이상_상주인구_수,hl_여성연령대_10_상주인구_수,hl_여성연령대_20_상주인구_수,hl_여성연령대_30_상주인구_수,hl_여성연령대_40_상주인구_수,hl_여성연령대_50_상주인구_수,hl_여성연령대_60_이상_상주인구_수,hl_총_가구_수,hl_아파트_가구_수,hl_비_아파트_가구_수
0,2019_2,1001010,28320,13999,14321,3720,4084,5078,4372,4783,...,2931,1870,2068,2431,2109,2491,3352,13046,1888,11158
1,2019_2,1001009,22242,10954,11288,2919,3124,3817,3404,3722,...,2411,1462,1598,1828,1622,1933,2845,10288,1770,8518
2,2019_2,1001008,16300,8141,8159,1647,2509,2931,2383,2789,...,1883,827,1307,1335,1115,1417,2158,8379,265,8114
3,2019_2,1001007,5262,2676,2586,768,806,675,749,1080,...,559,370,347,325,381,538,625,2175,182,1993
4,2019_2,1001006,21885,10817,11068,3442,3085,3294,3304,3824,...,2317,1597,1485,1628,1715,2024,2619,8598,3081,5517
